In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

In [ ]:
from keras.utils import load_img
from keras.utils import img_to_array
from keras.applications import vgg16
from IPython.display import display # 이미지 출력 함수
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
train_dir = '/gdrive/My Drive/Colab Notebooks/dataset/shoes_data/train'
validation_dir = '/gdrive/My Drive/Colab Notebooks/dataset/shoes_data/test'
batch_size = 32
image_size = 224

In [ ]:
#학습에 사용될 이미지 데이터 생성기, 이미지 데이터 증강
train_datagen = ImageDataGenerator(
    rotation_range = 180 , #회전 최대 20도
    width_shift_range = 0.2, #좌우 이동
    height_shift_range = 0.2, # 상하 이동
    horizontal_flip = True, # 좌우 반전
    vertical_flip=True, # 상하 반전
)

In [ ]:
# 검증에 사용될 이미지 데이터 생성기
validation_datagen = ImageDataGenerator()

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_size,image_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

In [ ]:
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(image_size, image_size),
    batch_size = batch_size,
    class_mode= 'categorical',
    shuffle = False)

In [ ]:
class_num=len(train_generator.class_indices)

custom_labels = list(validation_generator.class_indices.keys())

모델을 새롭게 정의

In [ ]:
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model
from keras import models
from keras import layers
from keras import optimizers
import keras.backend as K
from keras.applications import VGG16

In [ ]:
K.clear_session() # 새로운 세션으로 시작

#모델 불러오기
conv_layers = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
conv_layers.summary()

In [ ]:
# convolution layer를 학습되지 않도록 고정
for layer in conv_layers.layers:
    layer.trainable = False

In [ ]:
# 새로운 모델 생성하기
model = models.Sequential()

In [ ]:
# vgg16 모델의 convolution layer 추가
model.add(conv_layers)

In [ ]:
# 모델의 Fully connected 부분을 재구성
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(class_num, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# 모델 컴파일
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc'])

In [ ]:
from keras.models import load_model
vgg16_model_path = 'new_train_from_vgg16.h5'
model.save(vgg16_model_path)

In [ ]:
# 모델 학습
history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)

In [ ]:
model.save(vgg16_model_path)

In [ ]:
from google.colab import files

In [ ]:
files.download(vgg16_model_path)

In [ ]:
#학습률과 손실 확인

import matplotlib.pyplot as plt

acc = history.history['acc']
loss = history.history['loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='accuracy')
plt.plot(epochs, loss, 'r', label='loss')
plt.title('accuracy and loss')
plt.legend()
  
plt.show()

In [ ]:
from google.colab import files
files.download(vgg16_model_path)

저장시킨 모델로 예측하기

In [ ]:
from keras.models import load_model
vgg16_model_path = '/gdrive/My Drive/Colab Notebooks/new_train_from_vgg16.h5'

In [ ]:
model = load_model('/gdrive/My Drive/Colab Notebooks/new_train_from_vgg16.h5')

In [ ]:
# 분류코드 (기존 모델용)

def predict_vgg16(model, filename) :
    image = load_img(filename) #이미지 파일을 읽고 화면에 표시
    # image = PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=688x550
    display(image)
    
    image = load_img(filename, target_size=(224, 224)) #모델 사이즈로 이미지 파일 읽기
    # image = PIL.Image.Image image mode=RGB size=224x224
    
    image = img_to_array(image) #이미지 데이터를 numpy로 변환
    
    image = image.reshape((1, 224, 224, 3))
    #vgg16.preprocess_input()을 호출하기 위해 차원을 조정
    #보통 모델을 여러 이미지를 한번에 호출
    #맨앞의 1 = 이미지 갯수 1, 224 가로세로, 네번쨰 R,G,B 3개
    
    image = vgg16.preprocess_input(image)
    # Vgg16 모델 호출 위한 전처리
    # -255~255 사이 값으로 정규화
    # RGB를 BGR순으로 바꾼다
    
    yhat = model.predict(image) #이미지를 모델에 적용
    label = vgg16.decode_predictions(yhat) # 모델 적용된 결과를 파싱
    label = label[0][0] # 가장 확률이 높은 결과를 휙득
    print('%s (%.2f%%)' % (label[1], label[2]*100)) #라벨과 라벨을 예측한 확률을 출력

In [ ]:
def fe(filename) :
    image = load_img(filename) #이미지 파일을 읽고 화면에 표시
    # image = PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=688x550
    display(image)
    
    image = load_img(filename, target_size=(224, 224)) #모델 사이즈로 이미지 파일 읽기
    # image = PIL.Image.Image image mode=RGB size=224x224
    
    image = img_to_array(image) #이미지 데이터를 numpy로 변환
    
    image = image.reshape((1, 224, 224, 3))
    #vgg16.preprocess_input()을 호출하기 위해 차원을 조정
    #보통 모델을 여러 이미지를 한번에 호출
    #맨앞의 1 = 이미지 갯수 1, 224 가로세로, 네번쨰 R,G,B 3개
    
    image = vgg16.preprocess_input(image)
    # Vgg16 모델 호출 위한 전처리
    # -255~255 사이 값으로 정규화
    # RGB를 BGR순으로 바꾼다
    feature = model.predict(image)[0]
    return feature / np.linalg.norm(feature)

In [ ]:
features = []
img_paths = []

# Save Image Feature Vector with Database Images
for i in range(1, 3000):
    if i%100 == 0:
        print(i)
    try:
        image_path = '/gdrive/My Drive/Colab Notebooks/dataset/shoes_data/shoesCNN/' + str(i) + ".jpg"
        img_paths.append(image_path)

        # Extract Features
        feature = fe(image_path)

        features.append(feature)

        # Save the Numpy array (.npy) on designated path
        feature_path = '/gdrive/My Drive/Colab Notebooks/dataset/shoes_data/shoesCNN/' + str(i) + ".npy"
        np.save(feature_path, feature)
    except Exception as e:
        print('예외가 발생했습니다.', e)

In [ ]:
test_path = '/gdrive/My Drive/Colab Notebooks/random.jpg'

In [ ]:
# Extract its features
query = fe(test_path)

# Calculate the similarity (distance) between images
dists = np.linalg.norm(features - query, axis=1)

# Extract 30 images that have lowest distance
ids = np.argsort(dists)[:4]

scores = [(dists[id], img_paths[id], id) for id in ids]
# Visualize the result
axes=[]
fig=plt.figure(figsize=(8,8))
for a in range(4):
    score = scores[a]
    axes.append(fig.add_subplot(5, 6, a+1))
    subplot_title=str(round(score[0],2)) + str(score[2]+1)
    axes[-1].set_title(subplot_title)  
    plt.axis('off')
    plt.imshow(load_img(score[1]))
fig.tight_layout()
plt.show()